# 2. Grouping customers together!
Now we will deal with clustering algorithms that will provide groups of clients which are similar among them.

To solve this task, you must accomplish the following stages:

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import random

In [2]:
data = pd.read_csv('bank_transactions.csv')
data.head(10)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.00
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.00
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.00
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.00
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.50
5,T6,C1536588,8/10/72,F,ITANAGAR,53609.20,2/8/16,173940,676.00
6,T7,C7126560,26/1/92,F,MUMBAI,973.46,2/8/16,173806,566.00
7,T8,C1220223,27/1/82,M,MUMBAI,95075.54,2/8/16,170537,148.00
8,T9,C8536061,19/4/88,F,GURGAON,14906.96,2/8/16,192825,833.00
9,T10,C6638934,22/6/84,M,MUMBAI,4279.22,2/8/16,192446,289.11


In [3]:
data.shape

(1048567, 9)

In [4]:
# drop all missing values - they are not that many 
data.dropna(inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041614 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 79.5+ MB


In [6]:
# handling dates
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

In [7]:
data.TransactionTime = data.TransactionTime.apply(lambda x: datetime.strptime(str(x).zfill(6), '%H%M%S').time())

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041614 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   TransactionID            1041614 non-null  object        
 1   CustomerID               1041614 non-null  object        
 2   CustomerDOB              1041614 non-null  datetime64[ns]
 3   CustGender               1041614 non-null  object        
 4   CustLocation             1041614 non-null  object        
 5   CustAccountBalance       1041614 non-null  float64       
 6   TransactionDate          1041614 non-null  datetime64[ns]
 7   TransactionTime          1041614 non-null  object        
 8   TransactionAmount (INR)  1041614 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(5)
memory usage: 79.5+ MB


In [9]:
# shift the date of births with 100 to the left (for those with a DOB year > 2000)
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)

# drop the customers that have year = 1800
data.drop(data[data.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)

In [10]:
# save the pre-processed dataframe to a pickle file
data.to_pickle('data_prepr.pkl')

In [11]:
df1 = pd.read_pickle('data_prepr.pkl')
df1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5


# 2.1 Getting your data + feature engineering
2.Sometimes, the features (variables, fields) are not given in a dataset but can be created from it. The previous step is known as feature engineering. For example, the original dataset has several transactions done by the same customer. Then, we suggest you to group data by the client (using CustomerId) and, based on it, create the following new features for each CustomerId:

a) Number of transactions

b) Number of transactions with a balance bigger than 100 USD

c) Average amount of the transactions

d) Average balance

e) Average difference between the balance and the transaction amount for each customer (this is mainly known in the banking world as utilisation).

f) Most common gender of the customer

h) Most frequent location of the customer

So, in the end, you should have for each CustomerID seven features.

3.Consider at least 20 additional features that can be generated for each CustomerId. Describe each of them and why you believe they will be helpful. Moreover, add it to the previous dataset (the one with seven features). In the end, you should have for each CustomerID at least 27 features (7 recommended + 20 suggested by you).

Hints for feature engineering:

- Instead of only using the average, you could use other functions such as minimum, maximum, percentiles, etc.
- Think of adding filters to your features. For instance, in "Number of transactions with balance bigger than 100 USD”, 
  the filter was "bigger than 100 USD". Can you think about other filters, even including other variables?
- Think of including the information given by the fields transaction date and time in your calculations.
- Think about getting information from a customer’s birthday, even comparing it to other fields.
- You could also calculate percentages in your features. Think of a plausible set of percentages worth to be calculated.

In [12]:
# add a new feature CustomerAge = the age of the customer today
df1['CustAge'] = (( pd.to_datetime('today') - df1['CustomerDOB'] ) / np.timedelta64(1, 'Y')).round(0)
df1['CustAge'] = df1['CustAge'].astype(int)

In [13]:
# categorize the customers based on age
# young Age <= 25
# working Age>25 and Age<65
# retired Age>=65

# function that categorize a customer based on the age
def age_class(age):
    if age<=25:
        return 'young'
    elif ((age>25) and (age<65)):
        return 'working'
    else:
        return 'retired'

df1['Age_Class'] = df1['CustAge'].apply(lambda row: age_class(row))

In [14]:
# add a new feature that indicates if the balance > 100 USD
df1['Balance>100'] = np.where(df1['CustAccountBalance'] > 100, 1, 0)

In [15]:
# add a new feature that indicates if the transaction amount > 1000 USD
df1['Trans_Amnt>1000'] = np.where(df1['TransactionAmount (INR)'] > 1000, 1, 0)

In [16]:
# add a new feature Utilisation = difference between the balance and the transaction amount
df1['Utilisation'] = df1['CustAccountBalance'] - df1['TransactionAmount (INR)']

In [17]:
# create a variable that indicates the type of the customer based on the balance
# gold > 1,000,000 USD
# silver 
# bronze < 300,000 USD

def cust_type(balance):
    if balance > 1000000:
        return 'gold'
    elif balance < 300000:
        return 'bronze'
    else:
        return 'silver'

df1['Cust_Type'] = df1['CustAccountBalance'].apply(lambda row: cust_type(row))

In [18]:
# add a feature that indicates the month in whcih the person was born
df1['CustDOB_Month'] = df1['CustomerDOB'].apply(lambda row: row.month)

In [19]:
# feature that indicates the month when the transaction was made
df1['Trans_Month'] = df1['TransactionDate'].apply(lambda row: row.month)

# feature that indicates the day of the week when the transaction was made
df1['Trans_DayWeek'] = df1['TransactionDate'].apply(lambda row: row.dayofweek+1)

In [20]:
# function that checks if the day is on the weekend and returns 1 if it is, 0 oth.
def is_weekend(day):
    if (day == 6) or (day == 7):
        return 1
    return 0

# feature that indicates if the transaction was made on the weekend
df1['Trans_On_Wknd'] = df1['Trans_DayWeek'].apply(lambda row: is_weekend(row))

In [21]:
# function that converts the month to a season
def season(month):
    if ((month==12) or (month==1) or (month==2)):
        return 'winter'
    elif ((month==3) or (month==4) or (month==5)):
        return 'spring'
    elif ((month==6) or (month==7) or (month==8)):
        return 'summer'
    else:
        return 'autumn'
    
# feature that indicates the season when the transaction was made (based on the month)
df1['Trans_Season'] = df1['Trans_Month'].apply(lambda row: season(row))

In [22]:
# based on the time, we can see if the transaction was made in the morning [6-12), 
# during the day [12-17), in the evening [17-22), or during the night [22-6)

def trans_time(time):
    if (time.hour >= 6) and (time.hour < 12):
        return 'morning'
    elif (time.hour >= 12) and (time.hour < 17):
        return 'day'
    elif (time.hour >= 17) and (time.hour < 22):
        return 'evening'
    else:
        return 'night'

# feature that indicates when the transaction was made throughout the day
df1['Trans_MDEN'] = df1['TransactionTime'].apply(lambda row: trans_time(row))

In [23]:
df1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Balance>100,Trans_Amnt>1000,Utilisation,Cust_Type,CustDOB_Month,Trans_Month,Trans_DayWeek,Trans_On_Wknd,Trans_Season,Trans_MDEN
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,1,0,17794.05,bronze,10,2,1,0,winter,day
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,1,1,-25728.31,bronze,4,2,1,0,winter,day
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,1,0,17415.44,bronze,11,2,1,0,winter,day
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,1,1,864443.21,silver,9,2,1,0,winter,day
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,35,...,1,1,4951.93,bronze,3,2,1,0,winter,evening


In [24]:
# add a new feature that indicates whether the trans. amount is very large,large, medium, small
# very large VR, >5000 USD
# large L, >1000 and <=5000 USD
# medium M, >100 and <=1000 USD
# small S, <=100 USD

def trans_amnt(amount):
    if amount > 5000:
        return 'VL'
    elif ((amount > 1000) and (amount <= 5000)):
        return 'L'
    elif ((amount > 100) and (amount <= 1000)):
        return 'M'
    else:
        return 'S'

df1['Trans_Amnt_Type'] = df1['TransactionAmount (INR)'].apply(lambda row: trans_amnt(row))

In [25]:
# there are transactions of amount > current account balance
# we consider those as rejected transactions

# create a new feature that indicates if the transaction is rejected (Acc.Balance < Trans.Amnt) 
# or accepted (Acc.Balance > Trans.Amnt)
df1['Trans_Acc_Rej'] = np.where(df1['Utilisation'] < 0, 'rejected', 'accepted')

In [26]:
# ratio cust.acc.balance / total balance in the bank
total_bal = sum(df1['CustAccountBalance'])   # total balance across all accounts in the bank
df1['Ratio_AccBal'] = df1['CustAccountBalance'].apply(lambda row: row/total_bal)

In [27]:
# total account balance across all accounts with gender = F
total_bal_F = np.sum(df1[df1['CustGender'] == 'F']['CustAccountBalance'])

# total account balance across all accounts with gender = M
total_bal_M = np.sum(df1[df1['CustGender'] == 'M']['CustAccountBalance'])

df1['Ratio_AccBal_FM'] = np.where(df1['CustGender'] == 'F',df1['CustAccountBalance']/total_bal_F,df1['CustAccountBalance']/total_bal_M)

In [28]:
# total transaction amount across all accounts with gender = F
total_tramnt_F = np.sum(df1[df1['CustGender'] == 'F']['TransactionAmount (INR)'])

# total transaction amount across all accounts with gender = M
total_tramnt_M = np.sum(df1[df1['CustGender'] == 'M']['TransactionAmount (INR)'])

df1['Ratio_TrAmnt_FM'] = np.where(df1['CustGender'] == 'F',df1['TransactionAmount (INR)']/total_tramnt_F,df1['TransactionAmount (INR)']/total_tramnt_M)

In [29]:
df1.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Trans_Month,Trans_DayWeek,Trans_On_Wknd,Trans_Season,Trans_MDEN,Trans_Amnt_Type,Trans_Acc_Rej,Ratio_AccBal,Ratio_AccBal_FM,Ratio_TrAmnt_FM
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,2,1,0,winter,day,S,accepted,1.705100e-07,6.436753e-07,5.730795e-08
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,2,1,0,winter,day,VL,rejected,2.172817e-08,2.955814e-08,2.814347e-05
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,2,1,0,winter,day,M,accepted,1.710400e-07,6.456762e-07,1.052174e-06
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,2,1,0,winter,day,L,accepted,8.291546e-06,3.130059e-05,4.722175e-06
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5,35,...,2,1,0,winter,evening,L,accepted,6.425020e-08,2.425445e-07,4.040211e-06


In [30]:
# group the data by the Customer ID
df2 = df1.groupby('CustomerID').agg(Num_Trans = ('TransactionID', "count"),
                                    Num_Trans_Bal_100 = ('Balance>100', "sum"),
                                    Num_Trans_Amnt_1000 = ('Trans_Amnt>1000', "sum"),
                                    
                                    Avg_Amnt_Trans = ('TransactionAmount (INR)', "mean"),
                                    Max_Amnt_Trans = ('TransactionAmount (INR)', "max"),
                                    Min_Amnt_Trans = ('TransactionAmount (INR)', "min"),
                                    
                                    Avg_Bal = ('CustAccountBalance', "mean"),
                                    Max_Bal = ('CustAccountBalance', "max"),
                                    Min_Bal = ('CustAccountBalance', "min"),
                                    
                                    Avg_Util = ('Utilisation', "mean"),
                                    
                                    Acct_Age = ('CustAge','mean'),
                                    Acct_AgeCl = ('Age_Class', pd.Series.mode),
                                    
                                    Acct_Gender = ('CustGender', pd.Series.mode),
                                    Acct_Location = ('CustLocation', pd.Series.mode),
                                    
                                    Acct_CustType = ('Cust_Type', pd.Series.mode)).reset_index()

In [31]:
df2.head()

,CustomerID,Num_Trans,Num_Trans_Bal_100,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,C1010011,2,2,1,2553.0,4750.0,356.0,76340.635,120180.54,32500.73,73787.635,35.0,working,"[F, M]","[NEW DELHI, NOIDA]",bronze
1,C1010012,1,1,1,1499.0,1499.0,1499.0,24204.490,24204.49,24204.49,22705.490,28.0,working,M,MUMBAI,bronze
2,C1010014,2,2,1,727.5,1205.0,250.0,100112.950,161848.76,38377.14,99385.450,34.5,working,"[F, M]",MUMBAI,bronze
3,C1010018,1,1,0,30.0,30.0,30.0,496.180,496.18,496.18,466.180,33.0,working,F,CHAMPARAN,bronze
4,C1010024,1,1,1,5000.0,5000.0,5000.0,87058.650,87058.65,87058.65,82058.650,57.0,working,M,KOLKATA,bronze


In [32]:
df3 = df2.copy()

In [33]:
# when there is tie, pd.Series.mode returns a list of the most common values 
# we decided to choose on random one of those values
df3['Acct_Gender'] = df3['Acct_Gender'].apply(lambda row: random.choice(row))

In [34]:
def convert_list(info):
    if isinstance(info,str) == True:
        l = []
        l.append(info)
        return l
    return info

In [35]:
# convert all Location values to list of strings(locations)
df3['Acct_Location'] = df3['Acct_Location'].apply(lambda row: convert_list(row))
df3.head()

,CustomerID,Num_Trans,Num_Trans_Bal_100,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,C1010011,2,2,1,2553.0,4750.0,356.0,76340.635,120180.54,32500.73,73787.635,35.0,working,M,"[NEW DELHI, NOIDA]",bronze
1,C1010012,1,1,1,1499.0,1499.0,1499.0,24204.490,24204.49,24204.49,22705.490,28.0,working,M,[MUMBAI],bronze
2,C1010014,2,2,1,727.5,1205.0,250.0,100112.950,161848.76,38377.14,99385.450,34.5,working,M,[MUMBAI],bronze
3,C1010018,1,1,0,30.0,30.0,30.0,496.180,496.18,496.18,466.180,33.0,working,F,[CHAMPARAN],bronze
4,C1010024,1,1,1,5000.0,5000.0,5000.0,87058.650,87058.65,87058.65,82058.650,57.0,working,M,[KOLKATA],bronze


In [36]:
# when we have a tie, choose one of the values on random
df3['Acct_Location'] = df3['Acct_Location'].apply(lambda row: random.choice(row))

In [37]:
# convert Cust_Type to a list of strings(types)
df3['Acct_CustType'] = df3['Acct_CustType'].apply(lambda row: convert_list(row))

# when we have a tie, choose one of the values on random
df3['Acct_CustType'] = df3['Acct_CustType'].apply(lambda row: random.choice(row))

In [38]:
# convert Acct_AgeCl to a list of strings(types)
df3['Acct_AgeCl'] = df3['Acct_AgeCl'].apply(lambda row: convert_list(row))

# when we have a tie, choose one of the values on random
df3['Acct_AgeCl'] = df3['Acct_AgeCl'].apply(lambda row: random.choice(row))

In [39]:
df3.head()

,CustomerID,Num_Trans,Num_Trans_Bal_100,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,C1010011,2,2,1,2553.0,4750.0,356.0,76340.635,120180.54,32500.73,73787.635,35.0,working,M,NEW DELHI,bronze
1,C1010012,1,1,1,1499.0,1499.0,1499.0,24204.490,24204.49,24204.49,22705.490,28.0,working,M,MUMBAI,bronze
2,C1010014,2,2,1,727.5,1205.0,250.0,100112.950,161848.76,38377.14,99385.450,34.5,working,M,MUMBAI,bronze
3,C1010018,1,1,0,30.0,30.0,30.0,496.180,496.18,496.18,466.180,33.0,working,F,CHAMPARAN,bronze
4,C1010024,1,1,1,5000.0,5000.0,5000.0,87058.650,87058.65,87058.65,82058.650,57.0,working,M,KOLKATA,bronze


In [40]:
# save to pickle file
df3.to_pickle('data_CustID.pkl')

In [41]:
# data for each CustomerID (1 row per CustomerID)
df3 = pd.read_pickle('data_CustID.pkl')

In [42]:
# merge the big dataframe with the one that has specific info for each unique Cust ID

In [43]:
df_big = pd.merge(df1, df3)

In [44]:
df_big.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,25.0,17819.050,17819.05,17819.05,17794.050,28.0,working,F,JAMSHEDPUR,bronze
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,27999.0,2270.690,2270.69,2270.69,-25728.310,66.0,retired,M,JHAJJAR,bronze
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,459.0,17874.440,17874.44,17874.44,17415.440,26.0,working,F,MUMBAI,bronze
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,878.4,434849.495,866503.21,3195.78,433380.295,46.5,working,M,MUMBAI,bronze
4,T391139,C5342380,1979-05-29,M,MUMBAI,3195.78,2016-08-13,19:41:48,878.4,44,...,878.4,434849.495,866503.21,3195.78,433380.295,46.5,working,M,MUMBAI,bronze


In [45]:
# save to pickle file
df_big.to_pickle('data_big.pkl')

In [46]:
# data for each CustomerID (1 row per CustomerID)
df_final = pd.read_pickle('data_big.pkl')

# 2.2 Choose your features (variables)!
As you may notice, you have plenty of features to work with now. So, you need to find a way to reduce the dimensionality (reduce the number of variables to work with). You can follow the subsequent directions to achieve it:

1. To normalise or not to normalise? That's the question. Sometimes it is worth normalising (scaling) the features. Explain if it is a good idea to perform any normalisation method. If you think the normalisation should be used, apply it to your data (look at the available normalisation functions in the scikit-learn library).

2. Select one method for dimensionality reduction and apply it to your data. Some suggestions are Principal Component Analysis, Multiple Correspondence Analysis, Singular Value Decomposition, Factor Analysis for Mixed Data, Two-Steps clustering. Make sure that the method you choose applies to the features you have or modify your data to be able to use it. Explain why you chose that method and the limitations it may have.

3. Apply the selected method(s) to your data. Ensure the chosen method retains > 70% of the total variance.

In [24]:
df_final.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0,28,...,25.0,17819.050,17819.05,17819.05,17794.050,28.0,working,F,JAMSHEDPUR,bronze
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0,66,...,27999.0,2270.690,2270.69,2270.69,-25728.310,66.0,retired,M,JHAJJAR,bronze
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0,26,...,459.0,17874.440,17874.44,17874.44,17415.440,26.0,working,F,MUMBAI,bronze
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0,49,...,878.4,434849.495,866503.21,3195.78,433380.295,46.0,working,F,MUMBAI,bronze
4,T391139,C5342380,1979-05-29,M,MUMBAI,3195.78,2016-08-13,19:41:48,878.4,43,...,878.4,434849.495,866503.21,3195.78,433380.295,46.0,working,F,MUMBAI,bronze


In [25]:
df_norm = df_final.copy()

#### Normalizing Data

Normalization of data is a type of Feature scaling and is only required when the data distribution is unknown or the data doesn't have Gaussian Distribution. This type of scaling technique is used when the data has a diversified scope and the algorithms on which the data are being trained do not make presumptions about the data distribution such as Artificial Neural Network.


Normalization in machine learning is the process of translating data into the range [0, 1] (or any other range) or simply transforming data onto the unit sphere.

For example, if one of the variables in the K-Nearest Neighbor, KNN, is in the 1000s and the other is in the 0.1s, the first variable will dominate the distance rather strongly. In this scenario, normalization and standardization might be beneficial.

When you don’t know the distribution of your data or when you know it’s not Gaussian, normalization is a smart approach to apply. Normalization is useful when your data has variable scales and the technique you’re employing, such as k-nearest neighbors and artificial neural networks, doesn’t make assumptions about the distribution of your data.

As we stated before, every dataset does not need to be normalized for machine learning. It is only required when the ranges of characteristics are different.

Similarly, the goal of normalization is to change the values of numeric columns in the dataset to a common scale, without distorting differences in the ranges of values. For machine learning, every dataset does not require normalization. It is required only when features have different ranges.

For example, consider a data set containing two features, age, and income(x2). Where age ranges from 0–100, while income ranges from 0–100,000 and higher. Income is about 1,000 times larger than age. So, these two features are in very different ranges. When we do further analysis, like multivariate linear regression, for example, the attributed income will intrinsically influence the result more due to its larger value. But this doesn’t necessarily mean it is more important as a predictor. So we normalize the data to bring all the variables to the same range.

In [10]:
from sklearn import preprocessing

In [11]:
scaler = preprocessing.MinMaxScaler()

In [28]:
# choose only the numeric columns
df_numeric = df_norm.select_dtypes(include='number')
col_names = df_numeric.columns
df_numeric.head()

,CustAccountBalance,TransactionAmount (INR),CustAge,Balance>100,Balance>1000,Trans_Amnt>1000,Utilisation,CustDOB_Month,Trans_Month,Trans_DayWeek,...,Num_Trans_Bal_1000,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age
0,17819.05,25.0,28,1,1,0,17794.05,10,2,1,...,1,0,25.0,25.0,25.0,17819.050,17819.05,17819.05,17794.050,28.0
1,2270.69,27999.0,66,1,1,1,-25728.31,4,2,1,...,1,1,27999.0,27999.0,27999.0,2270.690,2270.69,2270.69,-25728.310,66.0
2,17874.44,459.0,26,1,1,0,17415.44,11,2,1,...,1,0,459.0,459.0,459.0,17874.440,17874.44,17874.44,17415.440,26.0
3,866503.21,2060.0,49,1,1,1,864443.21,9,2,1,...,2,1,1469.2,2060.0,878.4,434849.495,866503.21,3195.78,433380.295,46.0
4,3195.78,878.4,43,1,1,0,2317.38,5,8,6,...,2,1,1469.2,2060.0,878.4,434849.495,866503.21,3195.78,433380.295,46.0


In [30]:
# normalize the numeric columns
df_norm_num = scaler.fit_transform(df_numeric)
df_norm_num = pd.DataFrame(df_norm_num, columns = col_names)
df_norm_num.head()

,CustAccountBalance,TransactionAmount (INR),CustAge,Balance>100,Balance>1000,Trans_Amnt>1000,Utilisation,CustDOB_Month,Trans_Month,Trans_DayWeek,...,Num_Trans_Bal_1000,Num_Trans_Amnt_1000,Avg_Amnt_Trans,Max_Amnt_Trans,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age
0,0.000155,0.000016,0.06,1.0,1.0,0.0,0.013173,0.818182,0.090909,0.000000,...,0.166667,0.00,0.000016,0.000016,0.000016,0.000155,0.000155,0.000155,0.013173,0.06
1,0.000020,0.017948,0.44,1.0,1.0,1.0,0.012800,0.272727,0.090909,0.000000,...,0.166667,0.25,0.017948,0.017948,0.017948,0.000020,0.000020,0.000020,0.012800,0.44
2,0.000155,0.000294,0.04,1.0,1.0,0.0,0.013170,0.909091,0.090909,0.000000,...,0.166667,0.00,0.000294,0.000294,0.000294,0.000155,0.000155,0.000155,0.013170,0.04
3,0.007532,0.001320,0.27,1.0,1.0,1.0,0.020437,0.727273,0.090909,0.000000,...,0.333333,0.25,0.000942,0.001320,0.000563,0.003780,0.007532,0.000028,0.016739,0.24
4,0.000028,0.000563,0.21,1.0,1.0,0.0,0.013040,0.363636,0.636364,0.833333,...,0.333333,0.25,0.000942,0.001320,0.000563,0.003780,0.007532,0.000028,0.016739,0.24


In [32]:
# assign the normalized data to the original dataframe
df_norm[col_names] = df_norm_num
df_norm

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,...,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,0.000155,2016-02-08,14:32:07,0.000016,0.06,...,0.000016,0.000155,0.000155,0.000155,0.013173,0.06,working,F,JAMSHEDPUR,bronze
1,T2,C2142763,1957-04-04,M,JHAJJAR,0.000020,2016-02-08,14:18:58,0.017948,0.44,...,0.017948,0.000020,0.000020,0.000020,0.012800,0.44,retired,M,JHAJJAR,bronze
2,T3,C4417068,1996-11-26,F,MUMBAI,0.000155,2016-02-08,14:27:12,0.000294,0.04,...,0.000294,0.000155,0.000155,0.000155,0.013170,0.04,working,F,MUMBAI,bronze
3,T4,C5342380,1973-09-14,F,MUMBAI,0.007532,2016-02-08,14:27:14,0.001320,0.27,...,0.000563,0.003780,0.007532,0.000028,0.016739,0.24,working,F,MUMBAI,bronze
4,T391139,C5342380,1979-05-29,M,MUMBAI,0.000028,2016-08-13,19:41:48,0.000563,0.21,...,0.000563,0.003780,0.007532,0.000028,0.016739,0.24,working,F,MUMBAI,bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985317,T1048561,C5028150,1980-05-24,M,PUNE,0.000004,2016-09-18,18:46:32,0.001923,0.21,...,0.001923,0.000004,0.000004,0.000004,0.012999,0.21,working,M,PUNE,bronze
985318,T1048562,C1034220,1992-11-19,M,BANGALORE,0.000011,2016-09-18,18:47:34,0.000962,0.08,...,0.000962,0.000011,0.000011,0.000011,0.013018,0.08,working,M,BANGALORE,bronze
985319,T1048563,C8020229,1990-08-04,M,NEW DELHI,0.000066,2016-09-18,18:48:24,0.000512,0.10,...,0.000512,0.000066,0.000066,0.000066,0.013079,0.10,working,M,NEW DELHI,bronze
985320,T1048564,C6459278,1992-02-20,M,NASHIK,0.000237,2016-09-18,18:37:34,0.000295,0.09,...,0.000295,0.000237,0.000237,0.000237,0.013251,0.09,working,M,NASHIK,bronze


#### Dimensionality Reduction - PCA

In [44]:
df_pca = df_norm.drop(['TransactionID','CustomerID'], axis=1)
df_pca

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustAge,Age_Class,Balance>100,...,Min_Amnt_Trans,Avg_Bal,Max_Bal,Min_Bal,Avg_Util,Acct_Age,Acct_AgeCl,Acct_Gender,Acct_Location,Acct_CustType
0,1994-10-01,F,JAMSHEDPUR,0.000155,2016-02-08,14:32:07,0.000016,0.06,working,1.0,...,0.000016,0.000155,0.000155,0.000155,0.013173,0.06,working,F,JAMSHEDPUR,bronze
1,1957-04-04,M,JHAJJAR,0.000020,2016-02-08,14:18:58,0.017948,0.44,retired,1.0,...,0.017948,0.000020,0.000020,0.000020,0.012800,0.44,retired,M,JHAJJAR,bronze
2,1996-11-26,F,MUMBAI,0.000155,2016-02-08,14:27:12,0.000294,0.04,working,1.0,...,0.000294,0.000155,0.000155,0.000155,0.013170,0.04,working,F,MUMBAI,bronze
3,1973-09-14,F,MUMBAI,0.007532,2016-02-08,14:27:14,0.001320,0.27,working,1.0,...,0.000563,0.003780,0.007532,0.000028,0.016739,0.24,working,F,MUMBAI,bronze
4,1979-05-29,M,MUMBAI,0.000028,2016-08-13,19:41:48,0.000563,0.21,working,1.0,...,0.000563,0.003780,0.007532,0.000028,0.016739,0.24,working,F,MUMBAI,bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985317,1980-05-24,M,PUNE,0.000004,2016-09-18,18:46:32,0.001923,0.21,working,1.0,...,0.001923,0.000004,0.000004,0.000004,0.012999,0.21,working,M,PUNE,bronze
985318,1992-11-19,M,BANGALORE,0.000011,2016-09-18,18:47:34,0.000962,0.08,working,1.0,...,0.000962,0.000011,0.000011,0.000011,0.013018,0.08,working,M,BANGALORE,bronze
985319,1990-08-04,M,NEW DELHI,0.000066,2016-09-18,18:48:24,0.000512,0.10,working,1.0,...,0.000512,0.000066,0.000066,0.000066,0.013079,0.10,working,M,NEW DELHI,bronze
985320,1992-02-20,M,NASHIK,0.000237,2016-09-18,18:37:34,0.000295,0.09,working,1.0,...,0.000295,0.000237,0.000237,0.000237,0.013251,0.09,working,M,NASHIK,bronze


In [45]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(df_norm)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

ValueError: could not convert string to float: 'T1'

In [ ]:
# !pip install prince
import prince
import pprint

In [46]:
## Instantiate FAMD object
famd = prince.FAMD(
     n_components=25,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='auto',       ## Can be "auto", 'sklearn', 'fbpca'
     random_state=42)

## Fit FAMD object to data 
famd = famd.fit(df_pca) ## Exclude target variable "Churn"

## Inspect principal dimensions
pp = pprint.PrettyPrinter()
pp.pprint(famd.explained_inertia_) 

ValueError: Not all columns in "Categorical" group are of the same type